In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize
from torch.utils.data import Dataset
from torch.utils.data import random_split, Subset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        image = image.float()

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
def train_test_split(all_data, test_portion = 0.3, seed = 0):
  training_size = int((1 - test_portion) * len(all_data))
  test_size = len(all_data) - training_size
  training_indices, test_indices = random_split(
                              range(len(all_data)), 
                              [training_size, test_size],
                              generator=torch.Generator().manual_seed(seed))
  training_data = Subset(all_data, training_indices)
  test_data = Subset(all_data, test_indices)
  return training_data, test_data

In [ ]:
all_data = CustomImageDataset(
    annotations_file="/content/drive/MyDrive/Ai builders/annotations_file.csv", 
    img_dir="/content/drive/MyDrive/Ai builders/pig pics cleaned", 
    transform=Resize(size=256), 
    target_transform=None
)

training_data, test_data = train_test_split(all_data)

print("All data size: " + str(len(all_data)))
print("Training size: " + str(len(training_data)) + "\tTest size: " + str(len(test_data)))

All data size: 511
Training size: 357	Test size: 154


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 256, 256])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256*256*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=196608, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 8.870892  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss: 599577501630464.000000 

Epoch 2
-------------------------------
loss: 541208627642368.000000  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 3
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 4
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 5
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 6
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 7
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Epoch 8
-------------------------------
loss:     nan  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss:      nan 

Ep

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "healthy",
    "sick",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x.view(1, -1))
    
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "healthy", Actual: "sick"


In [ ]:
acc

106

In [ ]:
acc = 0
for i in range(len(test_data)):
  model.eval()
  x, y = test_data[i][0], test_data[i][1]
  with torch.no_grad():
    pred = model(x.view(1, -1))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'{i}, Predicted: "{predicted}", Actual: "{actual}"')
    if predicted == actual:
      acc += 1

0, Predicted: "healthy", Actual: "sick"
1, Predicted: "healthy", Actual: "healthy"
2, Predicted: "healthy", Actual: "sick"
3, Predicted: "healthy", Actual: "healthy"
4, Predicted: "healthy", Actual: "healthy"
5, Predicted: "sick", Actual: "healthy"
6, Predicted: "healthy", Actual: "healthy"
7, Predicted: "healthy", Actual: "sick"
8, Predicted: "healthy", Actual: "healthy"
9, Predicted: "healthy", Actual: "healthy"
10, Predicted: "healthy", Actual: "healthy"
11, Predicted: "healthy", Actual: "healthy"
12, Predicted: "healthy", Actual: "healthy"
13, Predicted: "healthy", Actual: "healthy"
14, Predicted: "healthy", Actual: "healthy"
15, Predicted: "healthy", Actual: "healthy"
16, Predicted: "healthy", Actual: "healthy"
17, Predicted: "healthy", Actual: "healthy"
18, Predicted: "healthy", Actual: "healthy"
19, Predicted: "healthy", Actual: "healthy"
20, Predicted: "healthy", Actual: "healthy"
21, Predicted: "healthy", Actual: "healthy"
22, Predicted: "healthy", Actual: "sick"
23, Predicted